In [1]:
import tensorflow as tf
cifar10 = tf.keras.datasets.cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)


170508288/170498071 [==============================] - 4s 0us/step


In [2]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x_train,y_train,test_size=0.2, random_state=42)

In [3]:
import albumentations as albu


def augment_image(augmentation, image):
    
    """
    Uses the Albumentations library.
    
    Inputs: 
    1. augmentation - this is the instance of type of augmentation to do 
    e.g. aug_type = HorizontalFlip(p=1) 
    # p=1 is the probability of the transform being executed.
    
    2. image - image with shape (h,w)
    
    Output:
    Augmented image as a numpy array.
    
    """
    # get the transform as a dict
    aug_image_dict =  augmentation(image=image)
    # retrieve the augmented matrix of the image
    image_matrix = aug_image_dict['image']
    return image_matrix

In [4]:
aug_types1 = albu.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=45, 
                  interpolation=1, border_mode=4, value=None, mask_value=None, 
                  shift_limit_x=None, shift_limit_y=None, always_apply=False, 
                  p=1)

aug_types2 = albu.Flip(p=1)

aug_types3 = albu.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=20, 
                  interpolation=1, border_mode=4, value=None, mask_value=None, 
                  shift_limit_x=None, shift_limit_y=None, always_apply=False, 
                  p=1)

aug_types4 = albu.Blur(blur_limit=(3,3.5), always_apply=False, p=1)

aug_types5 = albu.OneOf([
                albu.ElasticTransform(alpha=1, sigma=50, alpha_affine=50,
                                       interpolation=1, border_mode=4, value=None,mask_value=None,
                                       always_apply=False, approximate=False, p=1),
                albu.GridDistortion(num_steps=5, distort_limit=0.3, interpolation=1, border_mode=4, 
                                 value=None, mask_value=None, always_apply=False, p=1)
                        ], p=1)

In [5]:
import numpy as np
X_train = np.zeros((6*len(x_train),32,32,3))

In [6]:
for i in range(len(x_train)): 
  image=x_train[i]
  X_train[i]=image
  X_train[i+1*len(x_train)] = augment_image(aug_types1, image)
  X_train[i+2*len(x_train)] = augment_image(aug_types2, image)
  X_train[i+3*len(x_train)] = augment_image(aug_types3, image)
  X_train[i+4*len(x_train)] = augment_image(aug_types4, image)
  X_train[i+5*len(x_train)] = augment_image(aug_types5, image)
                

In [7]:
import pandas as pd
y_train = np.concatenate([y_train, y_train, y_train, y_train, y_train, y_train],axis=0)

In [8]:
from tensorflow.keras.applications.resnet50 import ResNet50
def preprocess_image_input(input_images):
  input_images = input_images.astype('float32')
  output_ims = tf.keras.applications.resnet50.preprocess_input(input_images)
  return output_ims


In [9]:
x_train = preprocess_image_input(X_train)
x_val = preprocess_image_input(x_val)
x_test = preprocess_image_input(x_test)

In [10]:
'''
Feature Extraction is performed by ResNet50 pretrained on imagenet weights. 
Input size is 224 x 224.
'''
def feature_extractor(inputs):

  feature_extractor = tf.keras.applications.resnet.ResNet50(input_shape=(224, 224, 3),
                                               include_top=False,
                                               weights='imagenet')(inputs)
  return feature_extractor


'''
Defines final dense layers and subsequent softmax layer for classification.
'''
def classifier(inputs):
    x = tf.keras.layers.GlobalAveragePooling2D()(inputs)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(1024, activation="relu")(x)
    x = tf.keras.layers.Dense(512, activation="relu")(x)
    x = tf.keras.layers.Dense(10, activation="softmax", name="classification")(x)
    return x

'''
Since input image size is (32 x 32), first upsample the image by factor of (7x7) to transform it to (224 x 224)
Connect the feature extraction and "classifier" layers to build the model.
'''
def final_model(inputs):

    resize = tf.keras.layers.UpSampling2D(size=(7,7))(inputs)

    resnet_feature_extractor = feature_extractor(resize)
    classification_output = classifier(resnet_feature_extractor)

    return classification_output

'''
Define the model and compile it. 
Use Stochastic Gradient Descent as the optimizer.
Use Sparse Categorical CrossEntropy as the loss function.
'''
def define_compile_model():
  inputs = tf.keras.layers.Input(shape=(32,32,3))
  
  classification_output = final_model(inputs) 
  model = tf.keras.Model(inputs=inputs, outputs = classification_output)
  initial_learning_rate =0.1
  lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=100000,
    decay_rate=0.96,
    staircase=True)
 
  model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=lr_schedule),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
  
  return model


model = define_compile_model()

model.summary()

2022-11-13 02:50:37.765907: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-13 02:50:37.889665: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-13 02:50:37.890619: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-13 02:50:37.892564: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

94781440/94765736 [==============================] - 1s 0us/step
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
up_sampling2d (UpSampling2D) (None, 224, 224, 3)       0         
_________________________________________________________________
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              2098176   
______________________________________________________________

In [11]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rescale=1./255)

datagen.fit(x_train)
datagen.fit(x_val)
datagen.fit(x_test)

In [12]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=10,
                          verbose=1)    

mc = ModelCheckpoint(filepath='best_model5.h5',
                     monitor='val_accuracy',
                     mode='max',
                     verbose=1,
                     save_weights_only=True,
                    save_best_only=True)
# fit the model
history =model.fit(datagen.flow(x_train, y_train, batch_size=32),
         
          epochs=30, verbose=1, validation_data=datagen.flow(x_val, y_val,
         batch_size=8), callbacks=[mc, earlystop])

2022-11-13 02:50:50.560197: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/30


2022-11-13 02:50:56.338263: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


7500/7500 [==============================] - 1254s 166ms/step - loss: 0.5880 - accuracy: 0.7971 - val_loss: 0.3227 - val_accuracy: 0.8913

Epoch 00001: val_accuracy improved from -inf to 0.89130, saving model to best_model5.h5
Epoch 2/30
7500/7500 [==============================] - 1241s 166ms/step - loss: 0.3243 - accuracy: 0.8866 - val_loss: 0.2236 - val_accuracy: 0.9287

Epoch 00002: val_accuracy improved from 0.89130 to 0.92870, saving model to best_model5.h5
Epoch 3/30
7500/7500 [==============================] - 1243s 166ms/step - loss: 0.2376 - accuracy: 0.9167 - val_loss: 0.2216 - val_accuracy: 0.9365

Epoch 00003: val_accuracy improved from 0.92870 to 0.93650, saving model to best_model5.h5
Epoch 4/30
7500/7500 [==============================] - 1243s 166ms/step - loss: 0.1919 - accuracy: 0.9326 - val_loss: 0.2183 - val_accuracy: 0.9396

Epoch 00004: val_accuracy improved from 0.93650 to 0.93960, saving model to best_model5.h5
Epoch 5/30
7500/7500 [============================

In [13]:
model = define_compile_model()
model.load_weights('best_model5.h5')
test_loss, test_acc = model.evaluate(datagen.flow(x_test, y_test, batch_size=32), verbose=2)
print(' Test_loss: %.3f test_accuracy:%.3f' % (test_loss, test_acc))

313/313 - 18s - loss: 0.2696 - accuracy: 0.9413
 Test_loss: 0.270 test_accuracy:0.941
